# Answering Business Questions using SQL

In this project, I will be using SQL to answer business questions about the [Chinook](https://github.com/lerocha/chinook-database) database.  I will use Python's SQLite module, as well as pandas to write queries that produce dataframe outputs. This will allow me to display tables in Jupyter. 

## Function Definitions

First, I will define three functions:
* `run_query`: this function takes a SQL query as an argument and returns a pandas dataframe of that query.
* `run_command`: this function takes a SQL command as an argument and executes it using the SQLite module. Since these queries don't return tables, we don't need to use pandas.
* `show_tables`: this function calls the `run_query` function to return a list of all tables and views in the database.
    

In [1]:
import sqlite3
import pandas as pd

database = 'chinook.db'

def run_query(query):
    with sqlite3.connect(database) as conn:
        return pd.read_sql(query, conn)
    
def run_command(command):
    with sqlite3.connect(database) as conn:
        conn.isolation_level = None
        conn.execute(command)
        
def show_tables():
    query = '''
    select 
        name, 
        type 
    from 
        sqlite_master 
    where 
        type in ("table", "view");
    '''
    return run_query(query)

In [2]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## Genre Decision

Now that those functions have been defined, the first prompt I will respond to is the following:

The Chinook record store has just signed a deal with a new record label, and I have been tasked with selecting the first three albumns that will be added to the store, from a list of four. The list I currently have appears as follows:

| __Artist Name__ | __Genre__ | 
|----|-----|
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

I will be determining which genres sell best in the USA, since that is where we will be advertising. 

I will create a query to find out which genres sell the most tracks in the USA, and then create a visualization of that data using pandas. 

In [3]:
query = '''
    with genre_num as (
        select 
            g.name as genre,
            c.country as country,
            sum(il.quantity) as num_sold
        from 
            genre g 
            left join track t on g.genre_id = t.genre_id
            left join invoice_line il on t.track_id = il.track_id
            left join invoice i on il.invoice_id = i.invoice_id
            left join customer c on c.customer_id = i.customer_id
        where 
            c.country = "USA"
        group by 1
    ),
    
    tot as (
    select 
        country,
        cast(sum(num_sold) as float) as overall
    from 
        genre_num
    group by 1
    )
    
    select 
        genre_num.genre as "Genre",
        genre_num.num_sold as "Number of Tracks Sold",
        genre_num.num_sold/tot.overall as "Percentage of Tracks Sold in the USA"       
    from 
        genre_num
        left join tot on genre_num.country = tot.country
    group by 1
    order by 2 desc
    limit 10
    ;
    '''

In [4]:
run_query(query)

,Genre,Number of Tracks Sold,Percentage of Tracks Sold in the USA
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


Looking a the top ten genres in the USA based on percentage of tracks sold, it is clear that the Rock genre has a majority at just over 53%. Since we have four genre options presented above (Hip-Hop, Punk, Pop, and Blues), I will choose the top three genres in terms of numbers of tracks sold. These will include Punk (12.4%), Blues (3.4%), and Pop (2%). Therefore, the artists whose albums we should purchase are as follows: Red Tone, Slim Jim Bites, and Meteor and the Girls. 

## Sales Support Agent Performance

Next, I will determine if any sales support agent is performing either better or worse than the others by analyzing the purchases of customers belonging to each employee. Below is a query that finds the total dollar amount of sales assigned to each sales support agent within the company.

In [5]:
query = '''
    select 
        e.first_name || " " || e.last_name as "Sales Support Agent",
        sum(i.total) as "Total Sales",
        e.hire_date as "Hire Date"
    from
        customer c 
        left join employee e on c.support_rep_id = e.employee_id
        left join invoice i on i.customer_id = c.customer_id
    group by 1;
'''

In [6]:
run_query(query)

,Sales Support Agent,Total Sales,Hire Date
0,Jane Peacock,1731.51,2017-04-01 00:00:00
1,Margaret Park,1584.00,2017-05-03 00:00:00
2,Steve Johnson,1393.92,2017-10-17 00:00:00


I also chose to include the hire date of each Sales Support Agent, since employees who have been with the company longer have the opportunity to earn a higher dollar amount of sales. 

From what I can see, Jane Peacock began working about one month before Margaret Park and has sold about 150 dollars more. Steve Johnson, on the othet hand, began working about five months later than Margaret Park, and has sold only about 200 dollars less. The one-month headstart that Jane had on Margaret allowed her to sell 150 dollars more, while the five-month headstart that Margaret had on Steve allowed her to sell only about 200 dollars more. 

A possible explanation would be that more customers are buying music in the fall and winter more than in the spring and summer, which is why Sales Support Agents at a digital music shop would perform better when they start in the fall. 

## Sales by Country 

My next task will be to analyze the sales data for customers from each different country, using the country value from the customers table. I will calculate data for each country on the following:
* total number of customers
* total value of sales
* average value of sales per customer
* average order value 

Because there are a number of countries with only one customer, I will group these customers as "Other" in my analysis. 

In [7]:
query = '''
with country_count as (
    select 
        count(distinct(c.customer_id)) as num_customers,
        sum(i.total) as tot,
        case when 
            count(distinct(c.customer_id)) = 1 then "Other"
            else c.country
            end as nation,
        case when 
            count(distinct(c.customer_id)) = 1 then 1
            else 0
            end as sort,
        count(distinct(i.invoice_id)) as num_orders
    from 
        customer c 
        left join invoice i on c.customer_id = i.customer_id
    group by c.country
)
select 
    nation as "Country",
    sum(num_customers) as "# Customers",
    tot as "Total Sales",
    round(tot/sum(num_customers), 2) as "CLV",
    round(tot/num_orders, 2) as "Avg Order Value"
from
    country_count
group by 1
order by sort asc, 3 desc;
'''

In [8]:
run_query(query)

,Country,# Customers,Total Sales,CLV,Avg Order Value
0,USA,13,1040.49,80.04,7.94
1,Canada,8,535.59,66.95,7.05
2,Brazil,5,427.68,85.54,7.01
3,France,5,389.07,77.81,7.78
4,Germany,4,334.62,83.65,8.16
5,Czech Republic,2,273.24,136.62,9.11
6,United Kingdom,3,245.52,81.84,8.77
7,Portugal,2,185.13,92.56,6.38
8,India,2,183.15,91.57,8.72
9,Other,15,75.24,5.02,7.52


Based on the table above, the greatest number of sales and number of customers are from the USA, with Canada in second place. While this is the largest market currently, there are other countries with larger customer lifetime values and higher average order values. Most notably, customers in the Czech Republic actually have an average Customer Lifetime Value of 136.62 dollars, which is quite a bit higher than that of the USA. Additionally, the average order value of customers in the Czech Republic is 9.11 dollars, as opposed to 7.94 dollars in the USA. 

While I would recommend exploring the potential for growth mostly in the Czech Republic, other good options can be Germany and the United Kingdom, which both have a greater the number of customers than that of the Czech Republic, as well as higher average order values and customer lifetime values than that of the USA. 

## Considering Purchasing Strategy

Next, I will determine if a new purchasing strategy that management is considering will save them money. The new strategy is to purchase only the most populat tracks from each album for record companies, instead of a whole album. Currently, the store allows customers to purchase either a whole album or individual tracks. I will determine what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue. 

The following edge cases must be considered:

* Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.

* Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

Since we're concerned with maximizing revenue, I will ignore the first edge case. Since previous analysis showed that the second case does not often happen, I will ignore this edge case as well. 

Below is a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:

* Number of invoices
* Percentage of invoices

First, I will test what I will call the `num_tracks` CTE.

In [9]:
query = '''
    select 
        album_id,
        count(track_id) as num_tracks
    from 
        track
    group by 1
    order by 1;
'''

In [10]:
run_query(query)

,album_id,num_tracks
0,1,10
1,2,1
2,3,3
3,4,8
4,5,15
5,6,13
6,7,12
7,8,14
8,9,8
9,10,14


Next, I will test what I will be calling the `tracks_purchased` CTE.

In [11]:
query = '''
    select 
        il.invoice_id,
        t.album_id,
        count(distinct(il.track_id)) as num_purchased
    from 
        invoice_line il
        left join track t on t.track_id = il.track_id 
    group by 1,2
    order by 1;
'''

In [12]:
run_query(query)

,invoice_id,album_id,num_purchased
0,1,91,16
1,2,20,1
2,2,34,1
3,2,39,1
4,2,66,1
5,2,73,1
6,2,134,1
7,2,190,1
8,2,215,1
9,2,218,1


Since both of these queries run on their own and we can visualize their outputs, I will now create a query that determines if each invoice is an album purchase or not.

In [20]:
query = '''
with num_tracks as(
    select 
        album_id,
        count(track_id) as num
    from 
        track
    group by 1
    order by 1
),

tracks_purchased as(
    select 
        il.invoice_id as invoice_id,
        t.album_id as album_id,
        count(distinct(il.track_id)) as num_purchased
    from 
        invoice_line il
        left join track t on t.track_id = il.track_id 
    group by 1,2
    order by 1
),

final as (
    select         
        tracks_purchased.invoice_id as invoice_id,
        tracks_purchased.album_id as album_id,
        tracks_purchased.num_purchased as num_purch,
        num_tracks.num,
        case when
            tracks_purchased.num_purchased = num_tracks.num
            then "yes"
            else "no"
            end as album_purch
    from tracks_purchased
        left join num_tracks on tracks_purchased.album_id = num_tracks.album_id
)

select 
    invoice_id as "Invoice ID",
    album_purch as "Album Purchase"
from 
    final 
group by 1
order by 1; 
'''

In [21]:
run_query(query)

,Invoice ID,Album Purchase
0,1,yes
1,2,no
2,3,no
3,4,no
4,5,yes
5,6,no
6,7,no
7,8,no
8,9,no
9,10,no


In [34]:
query = '''
with num_tracks as(
    select 
        album_id,
        count(track_id) as num
    from 
        track
    group by 1
    order by 1
),

tracks_purchased as(
    select 
        il.invoice_id as invoice_id,
        t.album_id as album_id,
        count(distinct(il.track_id)) as num_purchased
    from 
        invoice_line il
        left join track t on t.track_id = il.track_id 
    group by 1,2
    order by 1
),

final as (
    select         
        tracks_purchased.invoice_id as invoice_id,
        tracks_purchased.album_id as album_id,
        tracks_purchased.num_purchased as num_purch,
        num_tracks.num as num_t,
        case when
            tracks_purchased.num_purchased = num_tracks.num
            then "yes"
            else "no"
            end as album_purch
    from tracks_purchased
        left join num_tracks on tracks_purchased.album_id = num_tracks.album_id
)

select 
    (select count(num_t)
     from final
     where album_purch = "yes") as num_yes,
    (select count(num_t)
     from final
     where album_purch = "no") as num_no,
     ((select cast(count(num_t) as float)
     from final
     where album_purch = "yes") / (
     (select cast(count(num_t) as float)
     from final
     where album_purch = "yes") + 
     (select cast(count(num_t) as float)
     from final
     where album_purch = "no"))) as perct_yes,
     ((select cast(count(num_t) as float)
     from final
     where album_purch = "no") / (
     (select cast(count(num_t) as float)
     from final
     where album_purch = "yes") + 
     (select cast(count(num_t) as float)
     from final
     where album_purch = "no"))) as perct_no
from final;
'''
    

In [35]:
run_query(query)

,num_yes,num_no,perct_yes,perct_no
0,208,2869,0.067598,0.932402
1,208,2869,0.067598,0.932402
2,208,2869,0.067598,0.932402
3,208,2869,0.067598,0.932402
4,208,2869,0.067598,0.932402
5,208,2869,0.067598,0.932402
6,208,2869,0.067598,0.932402
7,208,2869,0.067598,0.932402
8,208,2869,0.067598,0.932402
9,208,2869,0.067598,0.932402


I have found that just over 93% of invoices include songs that are not sold in albums. Therefore, my recommendation is that the Chinook store does not continue to buy full albums from record companies.